# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
create_keyspace_command = """CREATE KEYSPACE IF NOT EXISTS sparkify
                             WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""

try:
    session.execute(create_keyspace_command)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
set_keyspace_command = "USE sparkify"

try:
    session.execute(set_keyspace_command)
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

## In this case session_id and session_item_number are enough to \
## make a record unique for our request. \
## Our complete PRIMARY KEY is composed by session_id, session_item_number 
create_statement =  """CREATE TABLE IF NOT EXISTS song_data_by_session (session_id INT,
        session_item_number INT,
        artist_name TEXT,
        song_title TEXT,
        song_length DOUBLE,
        PRIMARY KEY ((session_id, session_item_number)))"""
session.execute(create_statement)

select_statement = "SELECT artist_name, song_title, song_length FROM song_data_by_session WHERE session_id = %s AND session_item_number =  %s"
                    

In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        query = """INSERT INTO song_data_by_session (session_id, session_item_number, artist_name, song_title, song_length) """
        query = query + "VALUES  (%s, %s, %s, %s, %s)"

        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## Add in the SELECT statement to verify the data was entered into the table
result_set = session.execute(select_statement, (338, 4))

for row in result_set:
    print (row)

Row(artist_name='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=495.3073)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [11]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

## In this case user_id and session_id are the COMPOUND PARTITION KEY  \
## this allows us to have a unique PRIMARY KEY for our query, but for this request we have to \
## order by session_item_number but not to query on that so we have to declare session_item_number as CLUSTERING KEY. \
## Our complete PRIMARY KEY is composed by user_id, session_id, session_item_number 
create_statement =  """CREATE TABLE IF NOT EXISTS song_user_data_by_user_and_session_data (user_id INT,
        session_id INT,
        session_item_number INT,
        artist_name TEXT,
        song_title TEXT,
        user_first_name TEXT,
        user_last_name TEXT,
        PRIMARY KEY ((user_id, session_id), session_item_number))"""
session.execute(create_statement)

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        query = """INSERT INTO song_user_data_by_user_and_session_data (user_id, session_id, session_item_number, artist_name, song_title,
        user_first_name, user_last_name ) """
        query = query + " VALUES  (%s, %s, %s, %s, %s, %s, %s)"
        
        session.execute(query, (int(line[10]), 
                    int(line[8]), int(line[3]), line[0], line[9],
                    line[1], line[4]))
            

select_statement = "SELECT artist_name, song_title, user_first_name, user_last_name FROM song_user_data_by_user_and_session_data WHERE user_id = %s AND session_id = %s"

result_set = session.execute(select_statement, (10, 182))

for row in result_set:
    print (row)   

Row(artist_name='Down To The Bone', song_title="Keep On Keepin' On", user_first_name='Sylvie', user_last_name='Cruz')
Row(artist_name='Three Drives', song_title='Greece 2000', user_first_name='Sylvie', user_last_name='Cruz')
Row(artist_name='Sebastien Tellier', song_title='Kilometer', user_first_name='Sylvie', user_last_name='Cruz')
Row(artist_name='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', user_first_name='Sylvie', user_last_name='Cruz')


In [12]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## In this case, song_title is the PARTITION KEY and user_id \
## is the CLUSTERING KEY, the request asks to retrieve the user name \
## by song title, so we have to set song_title as PARTITION KEY, but \
## more users can listen to the same song so we may have many INSERT with the \
## same key, Cassandra overwrites data with the same key so we need to add a CLUSTERING KEY \
## because we need to have a unique record but not to query on that. \
## Our complete PRIMARY KEY is composed by song_title, user_id 
create_statement =  """CREATE TABLE IF NOT EXISTS user_data_by_song_title (song_title TEXT, user_id INT,
        user_first_name TEXT,
        user_last_name TEXT,
        PRIMARY KEY ((song_title), user_id))"""
session.execute(create_statement)


with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        query = """INSERT INTO user_data_by_song_title (song_title, user_id, user_first_name, user_last_name) """
        query = query + " VALUES  (%s, %s, %s, %s)"

        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

select_statement = "SELECT user_first_name, user_last_name FROM user_data_by_song_title WHERE song_title = %s"

result_set = session.execute(select_statement, ('All Hands Against His Own', ))

for row in result_set:
    print (row)

Row(user_first_name='Jacqueline', user_last_name='Lynch')
Row(user_first_name='Tegan', user_last_name='Levine')
Row(user_first_name='Sara', user_last_name='Johnson')


### Drop the tables before closing out the sessions

In [13]:
drop_song_data_by_session =  "DROP TABLE song_data_by_session"
song_user_data_by_user_and_session_data = "DROP TABLE song_user_data_by_user_and_session_data"
user_data_by_song_title = "DROP TABLE user_data_by_song_title"

In [14]:
session.execute(drop_song_data_by_session)
session.execute(song_user_data_by_user_and_session_data)
session.execute(user_data_by_song_title)

### Close the session and cluster connection¶

In [15]:
session.shutdown()
cluster.shutdown()

In [16]:
print('Process Ended')

Process Ended
